## File Description
This explores an informed search strategy. In particular, Greedy Best-First Search and A* Search will be examined with source code.

In [1]:
import heapq

## Greedy Best-First Search

### 1. Process
*    Initialisation ('priority' queue, visited set)
*    Main loop
    1. Choose one node from priority queue (i.e., choose the smallest value of f(n))
    2. Check if the node is goal (if yes, call the function that output the path from an initial node to a goal node.)
    3. Check if the node is visited previously (if yes, it will be skipped. if not, the node will be added in the visited set).
    4. Expand the next nodes (i.e., the current node's children).

### 2. Advantages
*    It does not require more memory than A* Search

### 3. Disadvantages
*    Imcomplete
*    Not optimal solution

In [3]:
def greedy_best_first_search(tree, start, goal, heuristic):
    """
    Performs Greedy Best-First Search on a tree/graph.
    :param tree: A dictionary representing the tree/graph as an adjacency list.
    :param start: The starting node.
    :param goal: The goal node.
    :param heuristic: A dictionary mapping nodes to their heuristic values.
    :return: The path from start to goal if found, or None if not found.
    """
    # Initialise the priority queue with the start node
    frontier = [(heuristic[start], start)]  # (heuristic value, node)
    heapq.heapify(frontier)

    visited = set()  # A set to track visited nodes
    parent = {}  # To reconstruct the path

    while frontier:
        # Extract the node with the smallest heuristic value
        _, current = heapq.heappop(frontier)

        # Check if the current node is the goal
        if current == goal:
            return reconstruct_path(parent, start, goal)

        # Skip if the node has already been visited
        if current in visited:
            continue

        # Mark the current node as visited
        visited.add(current)

        # Expand the node's children
        for child in tree.get(current, []):
            if child not in visited:
                heapq.heappush(frontier, (heuristic[child], child))
                parent[child] = current

    # Return None if no path to the goal was found
    return None

def reconstruct_path(parent, start, goal):
    """
    Reconstructs the path from start to goal using the parent dictionary.
    :param parent: A dictionary mapping each node to its parent.
    :param start: The starting node.
    :param goal: The goal node.
    :return: A list representing the path from start to goal.
    """
    path = []
    current = goal
    while current != start:
        path.append(current)
        current = parent.get(current)
    path.append(start)
    path.reverse()
    return path

In [4]:
# Example tree represented as an adjacency list
example_tree = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F', 'G'],
    'D': [],
    'E': [],
    'F': [],
    'G': []
}

# Example heuristic values (lower is better)
example_heuristic = {
    'A': 7,
    'B': 6,
    'C': 4,
    'D': 5,
    'E': 2,
    'F': 1,
    'G': 3
}

# Define the start and goal nodes
start_node = 'A'
goal_node = 'F'

# Run the Greedy Best-First Search
result = greedy_best_first_search(example_tree, start_node, goal_node, example_heuristic)

# Display the result
print(f"Path from {start_node} to {goal_node}: {result}")

Path from A to F: ['A', 'C', 'F']


## A* Search

### 1. Process
*    Initialisation (priority queue, visited set, and **set for storing g(n)**)
*    Main loop
    1. Choose one node from priority queue (i.e., choose the smallest value of f(n) **including g(n)**.)
    2. Check if the node is goal (if yes, call the function that output the path from an initial node to a goal node.)
    3. Check if the node is visited previously (if yes, it will be skipped. if not, the node will be added in the visited set).
    4. Expand the next nodes (i.e., the current node's children). Meanwile, the algorithm calculate g(n) to update a priority queue to be able to find a best choice.

### 2. Advantages
*    Completeness
*    Optimal solution

### 3. Disadvantages
*    It requires more memory than Greedy Best-First Search since it calculate g(n) at each parent.

In [5]:
import heapq

def a_star_search(tree, start, goal, heuristic, edge_costs):
    """
    Performs A* search on a tree/graph.
    :param tree: A dictionary representing the graph as an adjacency list.
    :param start: The starting node.
    :param goal: The goal node.
    :param heuristic: A dictionary mapping nodes to heuristic values.
    :param edge_costs: A dictionary mapping (parent, child) pairs to edge costs.
    :return: The path from start to goal if found, or None if no path exists.
    """
    # Priority queue (min-heap) with (f(n), node)
    frontier = [(0 + heuristic[start], start)]  # (f(n), node)
    heapq.heapify(frontier)

    g_score = {start: 0}  # Cost from start to each node
    parent = {}  # To reconstruct the path
    visited = set()  # To keep track of visited nodes

    while frontier:
        # Extract the node with the smallest f(n)
        _, current = heapq.heappop(frontier)

        # If the current node is the goal, reconstruct and return the path
        if current == goal:
            return reconstruct_path(parent, start, goal)

        # Mark the current node as visited
        visited.add(current)

        # Expand the node's neighbours
        for neighbour in tree.get(current, []):
            if neighbour in visited:
                continue

            # Calculate the tentative g(n)
            tentative_g = g_score[current] + edge_costs.get((current, neighbour), float('inf'))

            # If the tentative g(n) is better, update the frontier and scores
            if neighbour not in g_score or tentative_g < g_score[neighbour]:
                g_score[neighbour] = tentative_g
                f_score = tentative_g + heuristic[neighbour]
                heapq.heappush(frontier, (f_score, neighbour))
                parent[neighbour] = current

    # Return None if no path is found
    return None


def reconstruct_path(parent, start, goal):
    """
    Reconstructs the path from start to goal using the parent dictionary.
    :param parent: A dictionary mapping each node to its parent.
    :param start: The starting node.
    :param goal: The goal node.
    :return: A list representing the path from start to goal.
    """
    path = []
    current = goal
    while current != start:
        path.append(current)
        current = parent.get(current)
    path.append(start)
    path.reverse()
    return path


In [6]:
# Example graph as an adjacency list
example_tree = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F', 'G'],
    'D': [],
    'E': [],
    'F': [],
    'G': []
}

# Example heuristic values (lower values are better)
example_heuristic = {
    'A': 7,
    'B': 6,
    'C': 4,
    'D': 5,
    'E': 2,
    'F': 1,
    'G': 3
}

# Example edge costs
example_edge_costs = {
    ('A', 'B'): 1,
    ('A', 'C'): 2,
    ('B', 'D'): 4,
    ('B', 'E'): 1,
    ('C', 'F'): 3,
    ('C', 'G'): 1
}

# Define the start and goal nodes
start_node = 'A'
goal_node = 'F'

# Run A* search
result = a_star_search(example_tree, start_node, goal_node, example_heuristic, example_edge_costs)

# Display the result
print(f"Path from {start_node} to {goal_node}: {result}")


Path from A to F: ['A', 'C', 'F']


## EOF